<a id="top"></a>
# Plotly per serie storiche — Quickstart

Plotly è utile quando una serie storica non deve solo *mostrare* un andamento, ma anche permettere di **navigare nel tempo** (zoom e intervalli) e **cambiare punto di vista** senza riscrivere codice.

Qui si resta sul minimo che serve nella pratica:
- `plotly.express` per partire in 30 secondi;
- un salto a `graph_objects` per aggiungere **range slider / range selector** e **pulsanti** con `updatemenus`.

Dataset: `px.data.stocks()` (già incluso in Plotly).


## Indice

- [Setup minimo](#setup)
- [Legenda: i 4 elementi chiave](#legenda)
- [Dataset: stocks](#dataset)
- [Primo grafico con Plotly Express](#px-first)
- [Confronto tra serie (wide-form)](#px-wide)
- [Figure e tracce: cosa produce `px`](#traces)
- [Un salto: slider e pulsanti (documentazione ufficiale)](#controls)
- [Esportare in HTML](#export)


<a id="setup"></a>
## Setup minimo

Link utili (documentazione ufficiale):
- Plotly Express: https://plotly.com/python/plotly-express/
- Line charts: https://plotly.com/python/line-charts/
- Time series: https://plotly.com/python/time-series/


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


<a id="legenda"></a>
## Legenda: i 4 elementi chiave

Plotly diventa molto più leggibile se separi sempre quattro pezzi:

1) **Dati**: un DataFrame con una colonna tempo (`datetime`) ordinata.

2) **Mapping**: quali colonne finiscono su `x`, `y` (e, quando serve, `color`).

3) **Tracce**: una `Figure` è fatta di una o più tracce (`fig.data`). In una time series la traccia tipica è `go.Scatter` con `mode='lines'`.

4) **Layout & controlli**: titolo, assi, hover, range slider, pulsanti… si modificano con `fig.update_layout(...)` e `fig.update_xaxes(...)`.

Per riferimento:
- `layout.updatemenus`: https://plotly.com/python/reference/layout/updatemenus/


<a id="dataset"></a>
## Dataset: stocks

`px.data.stocks()` è un dataset *wide-form*: una colonna tempo (`date`) e una colonna per ciascuna serie.

Nota pratica: i valori sono **indicizzati** (partono da 1). È comodo per confrontare andamenti relativi.

Riferimento (wide-form in Plotly Express): https://plotly.com/python/wide-form/


In [ ]:
df = px.data.stocks().copy()
df["date"] = pd.to_datetime(df["date"])
df.head()

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Nota importante**

Se `date` non è `datetime` o non è ordinato, la linea può produrre risultati confusi (salti avanti/indietro).

</div>

<a id="px-first"></a>
## Primo grafico con Plotly Express

Il primo obiettivo è avere una linea leggibile con un mapping esplicito (`x` e `y`).


In [ ]:
company = "GOOG"

fig = px.line(
    df,
    x="date",
    y=company,
    title=f"{company} (indicizzato)"
)
fig.show()

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Esercizio**

Cambia `company` con un’altra colonna del dataset (`AAPL`, `AMZN`, `MSFT`, …). Poi limita il grafico agli ultimi 12 mesi filtrando `df` prima di passarlo a `px.line`.

</div>

In [ ]:
# Grafico con rolling mean su grafico originale plotly express

company = "GOOG"
rolling_window = 4
# NB: attento ai valori mancanti all'inizio della serie
# Calcolo della media mobile come nuova colonna di un nuovo DataFrame
df_rolling = df[[ "date", company]].copy()
df_rolling[f"{company}_rolling"] = df_rolling[company].rolling(rolling_window).mean()
# gestisci i valori NaN iniziali (opzionale)
df_rolling = df_rolling.dropna().reset_index(drop=True)
fig = px.line(
    df_rolling,
    x="date",
    y=[company, f"{company}_rolling"],
    title=f"{company} con media mobile a {rolling_window} giorni"
)
fig.show()

<a id="px-wide"></a>
## Confronto tra serie (wide-form)

Con i dati wide-form puoi passare direttamente una lista di colonne a `y`.

Questo pattern è quello che torna più spesso quando hai una tabella con molte serie già allineate nel tempo.


In [ ]:
companies = ["AAPL", "AMZN", "MSFT"]

fig = px.line(
    df,
    x="date",
    y=companies,
    title="Confronto tra serie (wide-form)"
)
fig.show()

<div style="background-color:#f5f5f5; padding:10px; border-left:4px solid #999;">

**Esercizio**

Aggiungi o rimuovi una società dalla lista `companies`. Poi crea una versione `df_indexed` che riporti tutte le serie a base 100 alla prima data (es.: `value / value.iloc[0] * 100`) e ripeti il grafico.

</div>

<a id="controls"></a>
## Un salto: slider e range selector (documentazione ufficiale)

Qui si passa a `graph_objects` per usare il **range slider / range selector** sull'asse X, uno dei controlli più utili per navigare serie storiche lunghe.

- **Range slider / range selector**: https://plotly.com/python/range-slider/

Il codice segue lo stesso schema degli esempi ufficiali: si costruiscono le tracce, poi si aggiunge il controllo nel `layout`.

Il range selector offre pulsanti predefiniti (1m, 6m, 1y, all) per saltare rapidamente a intervalli comuni. Lo slider in basso permette di selezionare manualmente un intervallo qualsiasi.

In [ ]:
companies = ["AAPL", "AMZN", "MSFT"]

fig = go.Figure()
for c in companies:
    fig.add_trace(go.Scatter(x=list(df["date"]), y=list(df[c]), name=c))

# Range slider + range selector (schema ufficiale)
fig.update_layout(
    title_text="Stocks (indicizzato) — range slider",
    height=400,
    width=700,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all"),
            ]),
            bgcolor="lightgray",
            font=dict(size=10),
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
    margin=dict(t=80, b=80),
)

fig.show()

<a id="export"></a>
## Esportare in HTML

Per condividere il grafico interattivo senza Jupyter, l’export in HTML è la strada più semplice.

Documentazione: https://plotly.com/python/interactive-html-export/


In [ ]:
# Esempio: salva l'ultima figura (fig) in HTML
# Il file è auto-contenuto e apribile in browser.

# fig.write_html("plot_time_series.html", include_plotlyjs="cdn")
